In [1]:
!pip install mlflow

     |████████████████████████████████| 14.6 MB 79 kB/s 
     |████████████████████████████████| 56 kB 3.4 MB/s 
     |████████████████████████████████| 1.1 MB 44.6 MB/s 
     |████████████████████████████████| 170 kB 47.8 MB/s 
     |████████████████████████████████| 636 kB 43.3 MB/s 
     |████████████████████████████████| 79 kB 8.7 MB/s 
     |████████████████████████████████| 145 kB 64.5 MB/s 
     |████████████████████████████████| 75 kB 5.2 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158172 sha256=c51b4e59f80f72fe9500e354d736cb902f5eb8a9d83d592673ce1f4672762dd5
  Stored in directory: /root/.cache/pip/wheels/be/5d/0a/9e13f53f4f5dfb67cd8d245bb7cdffe12f135846f491a283e3
  Created wheel for databricks-cli: filename=databricks_cli-0.15.0-py3-none-any.whl size=105260 sha256=cd22c528fc182756d941c3f854d0af36781ce1de299afe5e16f201de45b169ce

In [2]:
!pip install pyngrok


     |████████████████████████████████| 745 kB 6.9 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=169d1ec042cb299d6dbbc31da56fdf8b5555a69c0ab1d6bf51ba0227ba88c91e
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [3]:
get_ipython().system_raw("mlflow ui --port 5000 &")

In [4]:
from pyngrok import ngrok


In [5]:
ngrok.kill()


In [6]:
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)


In [7]:
print("MLflow UI ", ngrok_tunnel.public_url)


MLflow UI  https://e01e-34-125-92-16.ngrok.io


In [8]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.sklearn

if __name__ == "__main__":

    mlflow.set_experiment(experiment_name="mlflow demo")
    
    training_data = pd.read_csv('storepurchasedata.csv')
    print("loaded training data")

    training_data.describe()
    mlflow.log_param("training percentage",60)

    mlflow.log_param("dataset shape",training_data.shape)
    
    X = training_data.iloc[:, :-1].values
    y = training_data.iloc[:,-1].values
    
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.4,random_state=0)
    
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    print("Completed Feature Scaling")
    
    from sklearn.neighbors import KNeighborsClassifier
    # minkowski is for ecledian distance
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    
    # Model training
    classifier.fit(X_train, y_train)
    print("Model trained")
   
    y_pred = classifier.predict(X_test)
    y_prob = classifier.predict_proba(X_test)[:,1]
    
    from sklearn.metrics import confusion_matrix
    
    cm = confusion_matrix(y_test, y_pred)
    
    from sklearn.metrics import accuracy_score
    
    model_accuracy = accuracy_score(y_test,y_pred)
    
    print(model_accuracy)
    
    mlflow.log_metric("accuracy", model_accuracy)
    mlflow.sklearn.log_model(classifier, "model")
    
    

INFO: 'mlflow demo' does not exist. Creating a new experiment
loaded training data
Completed Feature Scaling
Model trained
0.8125
